<a href="https://colab.research.google.com/github/piotr14prywatny/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt


In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils  import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

from tensorflow.keras.layers import Conv1D

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
%reload_ext tensorboard

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test =  pd.read_pickle('data/train.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
y_train.shape

(34799, 43)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data',(X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model


In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred )

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return  Sequential([
      Conv2D( filters=64, kernel_size=(3,3), activation='relu', input_shape = input_shape, padding='same'),
      Conv2D( filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.5) ,

      Conv2D( filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D( filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D( filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D( filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),
      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense (num_classes, activation='softmax'),
])


In [0]:
def get_model(params):
  return  Sequential([
      Conv2D( filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape, padding='same'),
      Conv2D( filters=32, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']) ,

      Conv2D( filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D( filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D( filters=128, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D( filters=128, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),
      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense (num_classes, activation='softmax'),
])


In [0]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 9s 32ms/step - loss: 2.8023 - accuracy: 0.2370 - val_loss: 1.5410 - val_accuracy: 0.4734
Epoch 2/5
272/272 [==============================] - 7s 24ms/step - loss: 1.3162 - accuracy: 0.5698 - val_loss: 0.6110 - val_accuracy: 0.8009
Epoch 3/5
272/272 [==============================] - 7s 24ms/step - loss: 0.7303 - accuracy: 0.7642 - val_loss: 0.4159 - val_accuracy: 0.8641
Epoch 4/5
272/272 [==============================] - 7s 24ms/step - loss: 0.5250 - accuracy: 0.8320 - val_loss: 0.2534 - val_accuracy: 0.9151
Epoch 5/5
272/272 [==============================] - 7s 24ms/step - loss: 0.4047 - accuracy: 0.8738 - val_loss: 0.1976 - val_accuracy: 0.9324


0.932383114457312

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
      )
  
  score = model.evaluate(X_test, y_test)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
      
  
  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

ERROR! Session/line number was not unique in database. History logging moved to new session 63


In [0]:
space = {
    'batch_size' : hp.quniform('batch_size',100, 200, 10) ,

    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin (
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

Streaming output truncated to the last 5000 lines.
 984/1088 [==========================>...]
 - ETA: 0s - loss: 0.2218 - accuracy: 0.9245

 995/1088 [==========================>...]
 - ETA: 0s - loss: 0.2196 - accuracy: 0.9253

1006/1088 [==========================>...]
 - ETA: 0s - loss: 0.2197 - accuracy: 0.9252

1015/1088 [==========================>...]
 - ETA: 0s - loss: 0.2188 - accuracy: 0.9256

1025/1088 [===========================>..]
 - ETA: 0s - loss: 0.2182 - accuracy: 0.9257

1035/1088 [===========================>..]
 - ETA: 0s - loss: 0.2171 - acc